This following programm organises the data from the inputdataset in a way, that it can be used with the pytred Relevance Evaluator. This programm rankes the fascets according to the score of the crossencoder Model

In [1]:
import pandas as pd
import json
import pytrec_eval

In [2]:
dataset_file = '../../Data/Crossencoder/DF_Constructed_CrossEncoder_dataset_sc.csv'

In [3]:
df = pd.read_csv(dataset_file)

In [5]:
df =  df.sort_values('Unnamed: 0')

In [6]:
gk = df.groupby('query')

In [7]:
df

,Unnamed: 0.1,Unnamed: 0,query,clariQ,fascet,isRelated,score
189476,189476,0,rimouski,What do you want to know about the place?,climate,1,0.977413
193575,193575,1,rimouski,What do you want to know about the place?,economy,1,0.976314
232530,232530,2,rimouski,What do you want to know about the place?,municipal council,1,0.009128
62465,62465,3,rimouski,What do you want to know about the place?,history,1,0.987130
200856,200856,4,rimouski,What do you want to know about the place?,rimouski weather,0,0.954992
...,...,...,...,...,...,...,...
290737,290737,297147,eapw,"Which ""eapw"" do you mean?",eastern auto parts warehouse,0,0.997206
172604,172604,297148,eapw,"Which ""eapw"" do you mean?",usa,0,0.002049
47870,47870,297149,eapw,"Which ""eapw"" do you mean?",aftermarket auto parts,0,0.027946
163168,163168,297150,eapw,"Which ""eapw"" do you mean?",uk,0,0.001770


In [8]:
qrel = {}

for name, group in gk:
    queryidx = group.iloc[0]['Unnamed: 0']
    el_dict = {}
    for idx, row in group.iterrows():
        if(row['isRelated'] == 1):
            el_dict[row['fascet']] = row['isRelated']
    qrel['q'+str(queryidx)] = el_dict

run = {}
for name, group in gk:
    queryidx = group.iloc[0]['Unnamed: 0']
    el_dict = {}
    for idx, row in group.iterrows():
        el_dict[row['fascet']] = row['score']
    run['q'+str(queryidx)] = el_dict


In [9]:
qrel

{'q143608': {'2010': 1, '2013': 1, '2016': 1},
 'q284989': {'javascript': 1, 'python': 1},
 'q194133': {'windows 10': 1, 'windows 8': 1, 'windows 7': 1},
 'q164831': {'windows 10': 1,
  'windows 7': 1,
  'windows vista': 1,
  'server 2012': 1},
 'q228287': {'windows 10': 1,
  'windows 8': 1,
  'windows 7': 1,
  'windows vista': 1},
 'q15689': {'windows 8': 1,
  'windows vista': 1,
  'windows 7': 1,
  'windows xp': 1},
 'q69572': {'download': 1, 'update': 1},
 'q245607': {'windows 10': 1,
  'windows 8': 1,
  'windows 7': 1,
  'windows vista': 1},
 'q24388': {'windows 10': 1,
  'windows 8': 1,
  'windows 7': 1,
  'windows vista': 1},
 'q18185': {'esv': 1, 'kjv': 1, 'niv': 1, 'nlt': 1},
 'q226047': {'esv': 1, 'kjv': 1, 'niv': 1, 'nlt': 1},
 'q70559': {'esv': 1, 'kjv': 1, 'niv': 1},
 'q66588': {'esv': 1, 'kjv': 1, 'niv': 1, 'nlt': 1},
 'q84272': {'esv': 1, 'kjv': 1, 'niv': 1},
 'q259485': {'excel': 1, 'indesign': 1, 'word': 1},
 'q135048': {'tour': 1, 'albums': 1, 'lyrics': 1, 'members': 1

In [10]:
run

{'q143608': {'2010': 0.91648084,
  '2013': 0.24905457,
  '2016': 0.83908397,
  'windows 7': 0.0016564075,
  'md5': 0.9644408,
  'windows 8': 0.0016681467,
  'sha256': 0.001864255,
  'windows 10': 0.0017065671,
  'password': 0.0023002778,
  'spreadsheetguru': 0.06604918,
  '32': 0.777956,
  'mrexcel': 0.9957248},
 'q284989': {'javascript': 0.9907505,
  'python': 0.99400073,
  'java "%2f': 0.9798647,
  'windows 7': 0.0019459188,
  'characters': 0.0030322862,
  'windows 10': 0.0019293589,
  'html%2f': 0.9634322,
  'windows 8': 0.0020663992,
  'table': 0.018010983,
  'css%3f': 0.9805439,
  'readable format': 0.030368494,
  'javascript%2fs': 0.98005563},
 'q194133': {'windows 10': 0.99623626,
  'windows 8': 0.9963684,
  'windows 7': 0.99635905,
  'windows': 0.99604684,
  'mobile': 0.0020462954,
  'chrome': 0.0017365697,
  'store': 0.0016598952,
  'desktop': 0.9803104,
  'start menu': 0.01003693,
  'delete': 0.0019650047,
  'ui': 0.0020464007},
 'q164831': {'windows 10': 0.99623626,
  'windo

In [11]:
evaluator = pytrec_eval.RelevanceEvaluator(
    qrel, {'map', 'ndcg'})


In [13]:
evalResults = evaluator.evaluate(run)

In [14]:
sumMap = 0
sumNdcg = 0
for row in evalResults:
    sumMap = sumMap + evalResults[row]['map']
    sumNdcg = sumNdcg + evalResults[row]['ndcg']
    
    
print("Average Map Score:  " + str(sumMap/len(evalResults)))
print("Average NDCG Score: " + str(sumNdcg/len(evalResults)))

Average Map Score:  0.6115234825502529
Average NDCG Score: 0.7440013682609807


In [15]:
totalScores = {
    "map_sum": sumMap,
    "map_avg": sumMap/len(evalResults),
    "ndcg_sum": sumNdcg,
    "ndcg_avg": sumNdcg/len(evalResults),
}

In [16]:
with open('Output_CE_dataset_sc.json', 'w') as f:
    json.dump(evaluator.evaluate(run), f)

with open('Output_CE_dataset_sc_total.json', 'w') as f:
    json.dump(totalScores, f)